In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error




In [2]:
df_jan = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

### Q1. Downloading the data

```
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16
17
18
19




In [3]:
f"columns for jan data are {df_jan.shape[1]}"

'columns for jan data are 19'

### Q2. Computing duration
```
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59
42.59
52.59
62.59
```

In [4]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)



In [6]:
# Calculate fraction of remaining records
original_jan_count = len(df_jan)
original_feb_count = len(df_feb)

filtered_jan_count = len(df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)])
filtered_feb_count = len(df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)])

jan_fraction = filtered_jan_count / original_jan_count
feb_fraction = filtered_feb_count / original_feb_count

print(f"January fraction of records remaining: {jan_fraction:.3f}")
print(f"February fraction of records remaining: {feb_fraction:.3f}")

January fraction of records remaining: 0.981
February fraction of records remaining: 0.980


### Q4. One-hot encoding

```
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715
```

In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


df_jan[categorical] = df_jan[categorical].astype(str)

In [8]:
train_dicts = df_jan[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Number of columns in feature matrix: {X_train.shape[1]}")




Number of columns in feature matrix: 519


### Q5. Training a model

```
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64
```

In [9]:
train_dicts = df_jan[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Number of columns in feature matrix: {X_train.shape[1]}")


target = 'duration'
y_train= df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mse = mean_squared_error(y_train, y_pred)


#rmse = np.sqrt(mse)
rmse = root_mean_squared_error(y_train, y_pred)


print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Number of columns in feature matrix: 519
Mean Squared Error: 1765.0092805714291
Root Mean Squared Error: 42.01201352674529


### Q6. Evaluating the model

```
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81
```


In [11]:

val_dicts = df_feb[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
print(f"Number of columns in feature matrix: {X_val.shape[1]}")

target = 'duration'
y_val = df_feb[target].values


y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred)


rmse = np.sqrt(mse)
#rmse = root_mean_squared_error(X_val, y_pred)



print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Number of columns in feature matrix: 519
Mean Squared Error: 1878.0285030568737
Root Mean Squared Error: 43.336226220759855
